In [1]:
import pandas as pd
import gc

In [2]:
import numpy as np
import sys
import glob
sys.setrecursionlimit(9999)

In [3]:
csv_list = glob.glob('./HAI 2.0/training/*.csv')

In [4]:
train1 = pd.read_csv(csv_list[0]).rename(columns=lambda x: x.strip())
train2 = pd.read_csv(csv_list[1]).rename(columns=lambda x: x.strip())
train3 = pd.read_csv(csv_list[2]).rename(columns=lambda x: x.strip())

In [5]:
train = pd.concat([train1,train2,train3])
del train1 , train2, train3 
gc.collect()

0

In [6]:
TAG_MIN = train[['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79']].min()
TAG_MAX = train[['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79']].max()

In [7]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

In [8]:
train_scaled = normalize(train.drop('time', axis=1))
train_scaled = pd.DataFrame(train_scaled , columns=train.drop('time', axis=1).columns).ewm(alpha=0.9).mean()
train_scaled['time'] = train['time'].tolist()
train_scaled['time']= pd.to_datetime(train_scaled['time'])

In [9]:
from tqdm.notebook import trange
from datetime import timedelta
from tqdm import tqdm

import dateutil

valid_idxs = []
timestamp_list = train_scaled['time'].astype(str).tolist()
WINDOW_SIZE = 60

for L in trange(len(timestamp_list) - WINDOW_SIZE + 1):
    R = L + WINDOW_SIZE - 1
    if dateutil.parser.parse(timestamp_list[R]) - dateutil.parser.parse(timestamp_list[L]) == timedelta(seconds=WINDOW_SIZE - 1):
        valid_idxs.append(L)

In [10]:
tag_values = np.array(train_scaled.drop('time',axis=1), dtype=np.float32)
data_list = []
for q in valid_idxs[:-1] :
    item = {} 
    last = q + WINDOW_SIZE 
    item["given"] = tag_values[q : last]
    item["answer"] = tag_values[last]
    data_list.append(item)

In [11]:
X_train =  np.array([i['given'] for i in data_list ])
Y_train = np.array([i['answer'] for i in data_list ])

In [12]:
val_csv = pd.read_csv('./HAI 2.0/validation/validation.csv')
test_csv_list = glob.glob('./HAI 2.0/testing/*.csv')

In [13]:
test1 = pd.read_csv(test_csv_list[0]).rename(columns=lambda x: x.strip())
test2 = pd.read_csv(test_csv_list[1]).rename(columns=lambda x: x.strip())
test3 = pd.read_csv(test_csv_list[2]).rename(columns=lambda x: x.strip())
test4 = pd.read_csv(test_csv_list[3]).rename(columns=lambda x: x.strip())

test_df = pd.concat([test1,test2,test3,test4])

In [14]:
del test1 , test2, test3, test4 
gc.collect()

23

In [15]:
val_scaled = normalize(val_csv.drop(['time','attack'] , axis=1))
val_scaled = pd.DataFrame(val_scaled , columns=val_csv.drop(['time','attack'], axis=1).columns).ewm(alpha=0.9).mean()
val_scaled['time']= val_csv['time'].tolist()
val_scaled['time']= pd.to_datetime(val_scaled['time'])

In [16]:
valid_idxs_val = []
timestamp_list = val_scaled['time'].astype(str).tolist()

for L in trange(len(timestamp_list) - WINDOW_SIZE + 1):
    R = L + WINDOW_SIZE - 1
    if dateutil.parser.parse(timestamp_list[R]) - dateutil.parser.parse(timestamp_list[L]) == timedelta(seconds=WINDOW_SIZE - 1):
        valid_idxs_val.append(L)

In [17]:
tag_values_val = np.array(val_scaled.drop('time',axis=1), dtype=np.float32)
data_list_val = []
for q in valid_idxs_val[:-1] :
    item = {} 
    last = q + WINDOW_SIZE 
    item["given"] = tag_values_val[q : last]
    item["answer"] = tag_values_val[last]
    data_list_val.append(item)

In [18]:
X_val =  np.array([i['given'] for i in data_list_val ])
Y_val = np.array([i['answer'] for i in data_list_val ])


In [19]:
test_scaled = normalize(test_df.drop('time' , axis=1))
test_scaled = pd.DataFrame(test_scaled , columns=test_df.drop('time', axis=1).columns).ewm(alpha=0.9).mean()
test_scaled['time']= test_df['time'].tolist()
test_scaled['time']= pd.to_datetime(test_scaled['time'])

In [20]:
valid_idxs_test = []
timestamp_list = test_df['time'].astype(str).tolist()

for L in trange(len(timestamp_list) - WINDOW_SIZE + 1):
    R = L + WINDOW_SIZE - 1
    if dateutil.parser.parse(timestamp_list[R]) - dateutil.parser.parse(timestamp_list[L]) == timedelta(seconds=WINDOW_SIZE - 1):
        valid_idxs_test.append(L)

tag_values_test = np.array(test_scaled.drop('time',axis=1), dtype=np.float32)
data_list_test = []
for q in valid_idxs_test[:-1] :
    item = {} 
    last = q + WINDOW_SIZE 
    item["given"] = tag_values_test[q : last]
    item["answer"] = tag_values_test[last]
    data_list_test.append(item)

In [21]:
X_test =  np.array([i['given'] for i in data_list_test ])
Y_test = np.array([i['answer'] for i in data_list_test ])


In [26]:
import lightgbm as lgb
#from sklearn.multioutput import MultiOutputRegressor

In [27]:
X_train_reshape =X_train.reshape(len(X_train)  , WINDOW_SIZE * 79 )
X_val_reshape =X_val.reshape(len(X_val) , WINDOW_SIZE * 79 )
X_test_reshape = X_test.reshape(len(X_test) , WINDOW_SIZE * 79 )

del X_train , train, data_list, data_list_test , data_list_val, tag_values, tag_values_test, tag_values_val, val_csv, val_scaled, test_scaled, test_df 

gc.collect()



NameError: name 'X_train' is not defined

In [25]:
for i in trange(0,79) :
    model = lhb.XGBRegressor(random_state= 69, n_)

KeyboardInterrupt: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [27]:
dt = DecisionTreeClassifier()

In [28]:
dt

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [29]:
regressor_model

NameError: name 'regressor_model' is not defined

In [30]:
Y_val_predict = regressor_model.predict(X_val_reshape)

NameError: name 'regressor_model' is not defined

In [31]:
Y_val_predict.shape

NameError: name 'Y_val_predict' is not defined

In [32]:
error_val = abs(Y_val_predict - Y_val).sum(axis=1)

NameError: name 'Y_val_predict' is not defined

In [33]:
pd.Series(error_val).plot()

NameError: name 'error_val' is not defined

In [34]:
from joblib import dump, load
dump(regressor_model, 'lgb_60.joblib') 

NameError: name 'regressor_model' is not defined

In [35]:
label = pd.read_csv('./HAI 2.0/validation/validation.csv')['attack']

In [36]:
label[30:]

30       0
31       0
32       0
33       0
34       0
        ..
43196    0
43197    0
43198    0
43199    0
43200    0
Name: attack, Length: 43171, dtype: int64

In [37]:
error_val

NameError: name 'error_val' is not defined